In [2]:
%pip install nest_asyncio load_dotenv

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append('/Users/arthurj/Git_Repos/LightRAG')

In [4]:
import logging
import time
import json
import os
import pdfplumber
import nest_asyncio
from bs4 import BeautifulSoup
from load_dotenv import load_dotenv
from dataclasses import asdict, dataclass
from lightrag.lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete


In [5]:
nest_asyncio.apply()

In [6]:
# enviroment setup

WORKING_DIR = "/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory"
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

dotenv_path = '/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/.env'
load_dotenv(dotenv_path)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')

In [7]:
# model using 4o, text-embedding-3-small & moodys specific prompt to match banking inputs 

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=gpt_4o_complete
)

INFO:lightrag:Logger initialized for working directory: /Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory
INFO:lightrag:Load KV llm_response_cache with 0 data
INFO:lightrag:Load KV full_docs with 0 data
INFO:lightrag:Load KV text_chunks with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': '/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': '/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': '/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory/vdb_chunks.json'} 0 data


#### Ingestion

In [8]:
#set up ingesiton method for SnP data

def process_files_in_folder(folder_path):


    for filename in os.listdir(folder_path):
        folder = os.path.basename(folder_path)
        rag.logger.info(f"Processing folder: {folder}")
        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path) and filename.endswith('.txt') or filename.endswith('.html'):
           
            rag.set_current_file_info(folder, filename)

            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

                if filename.endswith('.html'):
                    soup = BeautifulSoup(content, 'html.parser')
                    content = soup.get_text()

                rag.insert(content)

            print(f"Processed: {filename}")
            rag.logger.info(f"Processed: {filename}")
      

In [9]:
ratings_file_path = "/Users/arthurj/Library/CloudStorage/OneDrive-moodys.com/Desktop/LightRag_Data/KG_data/Ratings"

process_files_in_folder(ratings_file_path)

INFO:lightrag:Processing folder: Ratings
INFO:lightrag:Processing file: Ratings/MERCK & CO., INC..txt
INFO:lightrag:Inserting content from Ratings/MERCK & CO., INC..txt
INFO:lightrag:[New Docs] inserting 1 docs
INFO:lightrag:Finished processing Ratings/MERCK & CO., INC..txt. Total tokens used: 0
INFO:lightrag:[New Chunks] inserting 2 chunks
INFO:lightrag:Inserting 2 vectors to chunks
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.44batch/s]
INFO:lightrag:Updated token count - Input: 1436, Output: 1436, Total: 2872
INFO:lightrag:Processing file: Ratings/MERCK & CO., INC..txt
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/2 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 616, Total: 6587
INFO:lightrag:Processing file: Ratings/MERCK & CO., INC..txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completi

Extracting entities from chunks:  50%|█████     | 1/2 [00:14<00:14, 14.49s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3549, Output: 304, Total: 18007
INFO:lightrag:Processing file: Ratings/MERCK & CO., INC..txt


Extracting entities from chunks: 100%|██████████| 2/2 [00:15<00:00,  7.57s/chunk]
INFO:lightrag:Inserting entities into storage...
Inserting entities: 100%|██████████| 19/19 [00:00<00:00, 5097.66entity/s]
INFO:lightrag:Inserting relationships into storage...
Inserting relationships: 100%|██████████| 8/8 [00:00<00:00, 4514.25relationship/s]
INFO:lightrag:Inserting 19 vectors to entities
Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.73s/batch]
INFO:lightrag:Inserting 8 vectors to relationships
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.08batch/s]
INFO:lightrag:Finished processing Ratings/MERCK & CO., INC..txt. Total tokens used: 18007
INFO:lightrag:Writing graph with 19 nodes, 8 edges
INFO:lightrag:Total tokens used after knowledge graph creation: 18007
INFO:lightrag:Processed: MERCK & CO., INC..txt
INFO:lightrag:Processing folder: Ratings
INFO:lightrag:Processing file: Ratings/ELI LILLY AND COMPANY.txt
INFO:lightrag:Inserting content from Ratings/ELI LIL

Processed: MERCK & CO., INC..txt


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.54batch/s]
INFO:lightrag:Updated token count - Input: 1521, Output: 1521, Total: 21049
INFO:lightrag:Processing file: Ratings/ELI LILLY AND COMPANY.txt
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/2 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 2984, Output: 395, Total: 24428
INFO:lightrag:Processing file: Ratings/ELI LILLY AND COMPANY.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 835, Total: 28362
INFO:lightrag:Processing file: Ratings/ELI LILLY AND COMPANY.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3406, Output: 431, Total: 32199
INFO:lightrag:Processing file: Ratings/ELI LILLY 

...

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.36batch/s]
INFO:lightrag:Updated token count - Input: 1924, Output: 1924, Total: 1802152
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/3 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 2225, Output: 276, Total: 1804653
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3098, Output: 718, Total: 1808469
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 2528, Output: 534, Total: 1811531
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt


Extracting entities from chunks:  33%|███▎      | 1/3 [00:17<00:35, 17.78s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3843, Output: 300, Total: 1815674
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt


Extracting entities from chunks:  67%|██████▋   | 2/3 [00:21<00:09,  9.39s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 683, Total: 1819456
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3809, Output: 394, Total: 1823659
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt


Extracting entities from chunks: 100%|██████████| 3/3 [01:51<00:00, 37.02s/chunk]
INFO:lightrag:Inserting entities into storage...
Inserting entities:   0%|          | 0/33 [00:00<?, ?entity/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 744, Output: 500, Total: 1824903
INFO:lightrag:Processing file: Ratings/RTX CORPORATION.txt
Inserting entities: 100%|██████████| 33/33 [00:10<00:00,  3.15entity/s]
INFO:lightrag:Inserting relationships into storage...
Inserting relationships: 100%|██████████| 17/17 [00:00<00:00, 6599.70relationship/s]
INFO:lightrag:Inserting 33 vectors to entities
Generating embeddings: 100%|██████████| 2/2 [00:01<00:00,  1.18batch/s]
INFO:lightrag:Inserting 17 vectors to relationships
Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.32s/batch]
INFO:lightrag:Finished processing Ratings/RTX CORPORATION.txt. Total tokens used: 1824903
INFO:lightrag:Writing graph with 1530

Processed: RTX CORPORATION.txt


In [11]:
recon_file_path = "/Users/arthurj/Library/CloudStorage/OneDrive-moodys.com/Desktop/LightRag_Data/KG_data/Recon"

process_files_in_folder(recon_file_path)

INFO:lightrag:Processing folder: Recon
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt
INFO:lightrag:Inserting content from Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt
INFO:lightrag:[New Docs] inserting 1 docs
INFO:lightrag:Finished processing Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt. Total tokens used: 1824903
INFO:lightrag:[New Chunks] inserting 32 chunks
INFO:lightrag:Inserting 32 vectors to chunks
Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.97s/batch]
INFO:lightrag:Updated token count - Input: 23051, Output: 23051, Total: 1871005
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/32 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated to

Extracting entities from chunks:   3%|▎         | 1/32 [00:17<09:11, 17.79s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 814, Total: 1917711
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3421, Output: 511, Total: 1921643
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt


Extracting entities from chunks:   6%|▋         | 2/32 [00:18<03:55,  7.87s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3287, Output: 624, Total: 1925554
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt


Extracting entities from chunks:   9%|▉         | 3/32 [00:18<02:06,  4.36s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 865, Total: 1929518
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3467, Output: 496, Total: 1933481
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt


Extracting entities from chunks:  12%|█▎        | 4/32 [00:19<01:21,  2.90s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3900, Output: 345, Total: 1937726
INFO:lightrag:Processing file: Recon/US5-26960522I_38e85743-0e01-3206-96fb-8f9aed7887c5_monitoring.txt


...

Extracting entities from chunks:  91%|█████████ | 31/34 [01:08<00:08,  2.78s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3915, Output: 631, Total: 8973801
INFO:lightrag:Processing file: Recon/BR01838723049991_40847a1c-bc6a-34fa-8a68-85f38957f29e_monitoring.txt


Extracting entities from chunks:  94%|█████████▍| 32/34 [01:09<00:04,  2.38s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3782, Output: 757, Total: 8978340
INFO:lightrag:Processing file: Recon/BR01838723049991_40847a1c-bc6a-34fa-8a68-85f38957f29e_monitoring.txt


Extracting entities from chunks:  97%|█████████▋| 33/34 [01:10<00:01,  1.89s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3804, Output: 488, Total: 8982632
INFO:lightrag:Processing file: Recon/BR01838723049991_40847a1c-bc6a-34fa-8a68-85f38957f29e_monitoring.txt


Extracting entities from chunks: 100%|██████████| 34/34 [01:20<00:00,  2.36s/chunk]
INFO:lightrag:Inserting entities into storage...
Inserting entities:   0%|          | 1/272 [00:00<00:32,  8.42entity/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 655, Output: 224, Total: 8983511
INFO:lightrag:Processing file: Recon/BR01838723049991_40847a1c-bc6a-34fa-8a68-85f38957f29e_monitoring.txt
Inserting entities:  96%|█████████▋| 262/272 [00:05<00:00, 50.37entity/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 649, Output: 224, Total: 8984384
INFO:lightrag:Processing file: Recon/BR01838723049991_40847a1c-bc6a-34fa-8a68-85f38957f29e_monitoring.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 1504, Output: 434, Total: 8986322
INFO:ligh

Processed: BR01838723049991_40847a1c-bc6a-34fa-8a68-85f38957f29e_monitoring.txt


In [ ]:
recon_file_path = "/Users/arthurj/Library/CloudStorage/OneDrive-moodys.com/Desktop/LightRag_Data/KG_data/Earnings_Call"

process_files_in_folder(recon_file_path)

INFO:lightrag:Processing folder: Earnings_Call
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:lightrag:Inserting content from Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:lightrag:[New Docs] inserting 1 docs
INFO:lightrag:Finished processing Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt. Total tokens used: 8995263
INFO:lightrag:[New Chunks] inserting 10 chunks
INFO:lightrag:Inserting 10 vectors to chunks
Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.39s/batch]
INFO:lightrag:Updated token count - Input: 9768, Output: 9768, Total: 9014799
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/10 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 2817, Output: 346, 

Extracting entities from chunks:  10%|█         | 1/10 [00:17<02:41, 17.95s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 967, Total: 9048222
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 989, Total: 9052310
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3099, Output: 729, Total: 9056138
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3728

Extracting entities from chunks:  20%|██        | 2/10 [00:21<01:15,  9.44s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3722, Output: 484, Total: 9064521
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt


Extracting entities from chunks:  30%|███       | 3/10 [00:21<00:37,  5.36s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 3896, Output: 339, Total: 9068756
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting entities from chunks:  40%|████      | 4/10 [00:22<00:20,  3.35s/chunk]INFO:lightrag:Updated token count - Input: 3853, Output: 413, Total: 9073022
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt


...

Extracting entities from chunks:  90%|█████████ | 9/10 [00:31<00:02,  2.46s/chunk]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 4115, Output: 986, Total: 9095528
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt


Extracting entities from chunks: 100%|██████████| 10/10 [00:39<00:00,  3.93s/chunk]
INFO:lightrag:Inserting entities into storage...
Inserting entities:   0%|          | 0/129 [00:00<?, ?entity/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 647, Output: 275, Total: 9096450
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
Inserting entities:  98%|█████████▊| 127/129 [00:06<00:00, 19.62entity/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 640, Output: 430, Total: 9097520
INFO:lightrag:Processing file: Earnings_Call/Coca-Cola Q3 2023 Earnings Call Transcript.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 712, Output: 461, Total: 9098693
INFO:lightrag:Processing file: Earnings_Call/C

Processed: Coca-Cola Q3 2023 Earnings Call Transcript.txt


Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.12s/batch]
INFO:lightrag:Updated token count - Input: 10208, Output: 10208, Total: 9119109
INFO:lightrag:Processing file: Earnings_Call/Abbott Laboratories Q2 2024 Earnings Call Transcript.txt
INFO:lightrag:[Entity Extraction]...
Extracting entities from chunks:   0%|          | 0/11 [00:00<?, ?chunk/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 2172, Output: 168, Total: 9121449
INFO:lightrag:Processing file: Earnings_Call/Abbott Laboratories Q2 2024 Earnings Call Transcript.txt
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:Updated token count - Input: 2367, Output: 251, Total: 9124067
INFO:lightrag:Processing file: Earnings_Call/Abbott Laboratories Q2 2024 Earnings Call Transcript.txt


Extracting entities from chunks:   9%|▉         | 1/11 [00:08<01:23,  8.35s/chunk]

In [ ]:
tenk_file_path = "/Users/arthurj/Library/CloudStorage/OneDrive-moodys.com/Desktop/LightRag_Data/KG_data/10K"

process_files_in_folder(tenk_file_path)

In [ ]:
tenq_file_path = "/Users/arthurj/Library/CloudStorage/OneDrive-moodys.com/Desktop/LightRag_Data/KG_data/10Q"

process_files_in_folder(tenq_file_path)

#### Visualization of Graph 

In [10]:
import networkx as nx
from pyvis.network import Network

graph_path = os.path.join(WORKING_DIR, 'graph_chunk_entity_relation.graphml')
G = nx.read_graphml(graph_path)

net = Network(notebook=True, cdn_resources="in_line")
net.from_nx(G)
output_file = os.path.join(WORKING_DIR, 'knowledge_graph.html')
net.show(output_file)

print(f"Interactive knowledge graph saved to: {output_file}")


/Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory/knowledge_graph.html
Interactive knowledge graph saved to: /Users/arthurj/Git_Repos/LightRAG/lightrag_expriments/notebooks/SnP_directory/knowledge_graph.html


#### Query Testing

In [ ]:
query_file_path = "/Users/arthurj/Git_Repos/ma-cpg-genai-research/graphrag-playground/lightrag/notebooks/example_queries.json"
output_file = os.path.join(WORKING_DIR, "../sales_documents/output_response.txt")

In [ ]:
def write_to_file(output_text):
   with open(output_file, "a", encoding="utf-8") as file:
       file.write(output_text + "\n")

In [ ]:
def load_queries_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as file:
        queries_data = json.load(file)
    return queries_data

In [ ]:
def process_queries_from_json(json_path):
    queries_data = load_queries_from_json(json_path)
    
    for item in queries_data:  
        query_number = item["example_number"]
        query_text = item["query"]
        
        for mode in ["hybrid"]: #"naive", "local", "global"
            response = rag.query(query_text, param=QueryParam(mode=mode))
            output_text = (
                f"Example Number: {query_number}, Mode: {mode}\n"
                f"Query: {query_text}\n"
                f"Response: {response}\n"
                "------------------------------------\n"
            )
            write_to_file(output_text)
            print(f"Processed Example Number: {query_number}, Mode: {mode}")

In [ ]:
process_queries_from_json(query_file_path)

#### Evaluation 

In [ ]:
# Evalutation method 